# Profiling and Optimizing NumPy in 10 Minutes

## David Wagner

1. Don't Guess
2. Don't Loop
3. Don't Copy
4. Don't Compile!
5. Don't JIT?
6. Don't Code at all!

# Don't Guess

In [2]:
import numpy as np
%load_ext line_profiler

In [3]:
x = np.random.randint(1000, size=(1000, 1000), dtype=np.int64)
y = np.random.randint(1000, size=(1000, 1000), dtype=np.int64)

def maybe_slow(x, y):
    add = x + y
    mult = x * y
    exp = x ** y
    return add + mult + exp

In [5]:
%lprun -f maybe_slow maybe_slow(x, y)

# Don't Loop

In [17]:
x

array([[558, 649, 373, ..., 586, 271, 405],
       [682, 219, 429, ..., 420, 751, 495],
       [684, 364, 975, ..., 172, 406, 521],
       ...,
       [939, 959, 296, ..., 170, 990, 260],
       [371, 411, 753, ..., 382, 703, 145],
       [196, 972,  67, ..., 530, 239, 625]])

In [19]:
x.shape

(1000, 1000)

In [20]:
rows, cols = x.shape
for i in range(rows):
    for j in range(cols):
        new[i, j] = x[i, j] + y[i, j]

NameError: name 'new' is not defined

In [ ]:
def definitely_slow(x, y)

    for i, j in 

# Don't Copy

In [21]:
def in_place_ops(x, y):
    new_array = x + y
    x += y
    return x

In [1]:
%lprun -u 0.001 -f in_place_ops [in_place_ops(x, y) for _ in range(1000)]

UsageError: Line magic function `%lprun` not found.


In [26]:
%%timeit 
in_place_ops(x, y)

2.43 ms ± 191 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Don't  Compile, JIT!

In [8]:
import numba
from numba import jit

In [12]:
import numexpr as ne

In [17]:
def maths_numpy(a, b):
    a *= b
    x1 = a / b
    a += b
    a -= b
    a %= b
    
@jit(nopython=True, cache=False, fastmath=False, parallel=True)
def maths_numba(a, b):
    a *= b
    x1 = a / b
    a += b
    a -= b
    a %= b
    
def maths_numexpr(a, b):
    ne.evaluate('a * b', out=a)
    x1 = ne.evaluate('a / b')
    ne.evaluate('a + b', out=a)
    ne.evaluate('a - b', out=a)
    ne.evaluate('a % b', out=a)
    
def maths_py(a, b):
    _ = [x * y for x, y in zip(a, b)]
    _ = [x / y for x, y in zip(a, b)]
    _ = [x + y for x, y in zip(a, b)]
    _ = [x - y for x, y in zip(a, b)]
    _ = [x % y for x, y in zip(a, b)]

In [18]:
def all_(a, b):
    x = maths_py(a, b)
    x = maths_numpy(a, b)
    x = maths_numba(a, b)
    x = maths_numexpr(a, b)
    return x

In [19]:
%lprun -u .001 -f all_ [all_(x, y) for _ in range(10)]

/Users/davidwagner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/Users/davidwagner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in remainder
/Users/davidwagner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/davidwagner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/davidwagner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in remainder
  
/Users/davidwagner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divi

# Don't JIT, Compile?

# Don't Reinvent The Wheel